In [2]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
for row in mi.rows:
    print(row.original_coordinates)

[5651, 230, 3450]
[5901, 650, 6670]
[7426, 245, 3840]
[4000, 200, 7800]
[4333, 224, 7068]
[0, 2000, 1600]
[4761, 4952, 6493]
[7000, 4151, 9017]
[4275, 7350, 5700]
[4443, 5716, 3322]


In [ ]:
from annotator_metrics.util.doc_io import MaskInformation
from annotator_metrics.src.preprocessing import copy_data
mi = MaskInformation()
print(mi.rows)
copy_data(group = "group1", output_base_path="/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/")

In [2]:
from annotator_metrics.util.doc_io import MaskInformation
mi = MaskInformation()
print(mi.rows[0].organelle_info)


{'mito_mem ': 3, 'mito_lumen ': 4, 'mito': 0}


In [3]:
import numpy as np
a=np.random.random((4,4,4))

In [7]:
print(a[1:3,1:3,1:3])

[[[0.29667459 0.14585225]
  [0.93038333 0.71463271]]

 [[0.04604659 0.92718152]
  [0.80627626 0.55996372]]]


In [1]:
from annotator_metrics.src.metrics_evaluations import calculate_all_to_all
results= calculate_all_to_all("group1","/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/", num_workers = 10)
print(results[1:10])

distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-o7sfg0c4', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-y1yqxem1', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-c4urnbca', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-ud7og50n', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/annotator-metrics/dask-worker-space/worker-6ka8ohzs', purging
distributed.diskutils - INFO - Found stale lock file and directory '/groups/scicompsoft/home/ackermand/Programming/

http://127.0.0.1:8787/status


In [5]:
import dask
from dask.distributed import Client
import tifffile
client = Client(n_workers = 1)
def test(i):
    if i%2 == 0:
        b=[tifffile.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/a1.tif'),tifffile.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/b1.tif')]
    else:
        b=[tifffile.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/a2.tif'),tifffile.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/b2.tif')]

    return b[0].shape
l = []
for i in range(100):
    l.append(dask.delayed(test)(i))

results = dask.compute(*l)
print(results)


/groups/scicompsoft/home/ackermand/miniconda3/envs/annotator-metrics/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38845 instead
  f"Port {expected} is already in use.\n"


((122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 185), (122, 186, 18

In [11]:
import time
import tifffile
from dask_image import imread
start = time.time(); temp = imread.imread('/groups/cosem/cosem/ackermand/annotation_and_analytics/tmp/group1/01/a1.tif'); print("loading: ", time.time() - start)
print(temp.shape)


loading:  0.008377313613891602
(122, 186, 185)


In [6]:
import dask
import dask.distributed
dask.config.set({"distributed.comm.timeouts.connect":100})
dask.config.get("distributed.comm.timeouts.connect")

100